### Installing required libraries

In [1]:
pip install hmmlearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [15]:
"""
Importing all the needed libraries for the project:
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
import statsmodels.api as sm1
from scipy.stats import linregress
import matplotlib.dates as mdates
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.model_selection  import PredefinedSplit, GridSearchCV
from sklearn.base import clone
import warnings
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import gdown
warnings.filterwarnings('ignore')

In [16]:
# Define Google Drive file IDs
prices_id       = "1vvp2zuaa8vuUCvp34yXFMubGmE460Wkw"
prices_train_id = "1x1IJzYs24RwpbOTpaNxuH-Z9OQ9Ch8jE"
prices_test_id  = "1v4JsvjWctSafjHm0JjXgpSCNLC-ZpwsE"

# Download and load prices
print("Loading prices...")
gdown.download(f"https://drive.google.com/uc?id={prices_id}", "prices.csv", quiet=False)
prices = pd.read_csv("prices.csv", parse_dates=True, index_col=0)
print("✓ Loaded prices")

# Download and load training set
print("Loading prices_train...")
gdown.download(f"https://drive.google.com/uc?id={prices_train_id}", "prices_train.csv", quiet=False)
prices_train = pd.read_csv("prices_train.csv", parse_dates=True, index_col=0)
print("✓ Loaded prices_train")

# Download and load test set
print("Loading prices_test...")
gdown.download(f"https://drive.google.com/uc?id={prices_test_id}", "prices_test.csv", quiet=False)
prices_test = pd.read_csv("prices_test.csv", parse_dates=True, index_col=0)
print("✓ Loaded prices_test")


Loading prices...


Downloading...
From: https://drive.google.com/uc?id=1vvp2zuaa8vuUCvp34yXFMubGmE460Wkw
To: /Users/mijitrenkel/Desktop/untitled folder 2/prices.csv
100%|██████████| 34.3M/34.3M [00:00<00:00, 37.3MB/s]


✓ Loaded prices
Loading prices_train...


Downloading...
From: https://drive.google.com/uc?id=1x1IJzYs24RwpbOTpaNxuH-Z9OQ9Ch8jE
To: /Users/mijitrenkel/Desktop/untitled folder 2/prices_train.csv
100%|██████████| 25.7M/25.7M [00:00<00:00, 39.7MB/s]


✓ Loaded prices_train
Loading prices_test...


Downloading...
From: https://drive.google.com/uc?id=1v4JsvjWctSafjHm0JjXgpSCNLC-ZpwsE
To: /Users/mijitrenkel/Desktop/untitled folder 2/prices_test.csv
100%|██████████| 8.59M/8.59M [00:00<00:00, 38.0MB/s]


✓ Loaded prices_test


In [17]:
prices.head()

,coin,open,high,low,close,volume,day,MVRV,n_unique_addresses,exchange_volume,...,cumret_regime_1,ret_regime_2,cumret_regime_2,is_regime_0,is_regime_1,is_regime_2,t1,tVal,bin,windowSize
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01 01:00:00,BTC,13791.4,13804.9,13680.0,13680.0,5.922596,2018-01-01,2.922937,838926.0,1.506584e+09,...,0.0,0.000000,0.000000,0,0,1,2018-01-01 08:00:00,3.067899,1.0,7
2018-01-01 02:00:00,BTC,13500.4,13600.0,13480.0,13514.0,9.326084,2018-01-01,2.922937,838926.0,1.506584e+09,...,0.0,-0.012209,-0.012209,0,0,1,2018-01-01 05:00:00,9.018793,1.0,3
2018-01-01 03:00:00,BTC,13512.0,13576.4,13512.0,13560.2,11.704644,2018-01-01,2.922937,838926.0,1.506584e+09,...,0.0,0.003413,-0.008796,0,0,1,2018-01-01 19:00:00,-3.649961,-1.0,16
2018-01-01 04:00:00,BTC,13694.8,13809.8,13667.4,13667.4,17.126073,2018-01-01,2.922937,838926.0,1.506584e+09,...,0.0,0.007874,-0.000921,0,0,1,2018-01-01 19:00:00,-4.418316,-1.0,15
2018-01-01 05:00:00,BTC,13763.3,13887.9,13658.6,13765.0,9.322753,2018-01-01,2.922937,838926.0,1.506584e+09,...,0.0,0.007116,0.006194,0,0,1,2018-01-01 19:00:00,-4.935372,-1.0,14


In [18]:
prices.tail()

,coin,open,high,low,close,volume,day,MVRV,n_unique_addresses,exchange_volume,...,cumret_regime_1,ret_regime_2,cumret_regime_2,is_regime_0,is_regime_1,is_regime_2,t1,tVal,bin,windowSize
date,,,,,,,,,,,,,,,,,,,,,
2021-12-30 20:00:00,BTC,47540.18,47584.01,47445.76,47515.82,1811.655,2021-12-30,2.072123,674082.0,2.653706e+08,...,0.58535,0.0,-0.821005,1,0,0,2021-12-31 16:00:00,4.789920,1.0,20
2021-12-30 21:00:00,BTC,47183.28,47238.85,46856.90,47094.59,3389.350,2021-12-30,2.072123,674082.0,2.653706e+08,...,0.58535,0.0,-0.821005,1,0,0,2021-12-31 16:00:00,5.577754,1.0,19
2021-12-30 22:00:00,BTC,47281.36,47313.78,47180.00,47206.67,1316.976,2021-12-30,2.072123,674082.0,2.653706e+08,...,0.58535,0.0,-0.821005,1,0,0,2021-12-31 16:00:00,5.372049,1.0,18
2021-12-30 23:00:00,BTC,47067.99,47182.99,46961.41,47012.64,2823.252,2021-12-30,2.072123,674082.0,2.653706e+08,...,0.58535,0.0,-0.821005,1,0,0,2021-12-31 16:00:00,5.381470,1.0,17
2021-12-31 00:00:00,BTC,47095.02,47366.95,47055.00,47313.98,2374.621,2021-12-31,2.072123,703405.0,2.653706e+08,...,0.58535,0.0,-0.821005,1,0,0,2021-12-31 16:00:00,4.930206,1.0,16


In [19]:
prices_train.tail()

,coin,open,high,low,close,volume,day,MVRV,n_unique_addresses,exchange_volume,...,cumret_regime_1,ret_regime_2,cumret_regime_2,is_regime_0,is_regime_1,is_regime_2,t1,tVal,bin,windowSize
date,,,,,,,,,,,,,,,,,,,,,
2020-12-31 19:00:00,BTC,28883.17,28912.01,28761.52,28836.48,1628.861,2020-12-31,3.132289,717073.0,7.510418e+08,...,0.534338,0.007374,-0.996634,0,0,1,2021-01-01 17:00:00,4.534670,1.0,22
2020-12-31 20:00:00,BTC,28917.08,28980.72,28902.61,28957.42,737.168,2020-12-31,3.132289,717073.0,7.510418e+08,...,0.534338,0.004185,-0.992449,0,0,1,2021-01-01 17:00:00,3.853311,1.0,21
2020-12-31 21:00:00,BTC,29155.96,29173.49,28915.96,29018.61,1894.056,2020-12-31,3.132289,717073.0,7.510418e+08,...,0.534338,0.002111,-0.990338,0,0,1,2021-01-01 17:00:00,3.318647,1.0,20
2020-12-31 22:00:00,BTC,28993.77,29066.92,28920.07,29009.63,1021.469,2020-12-31,3.132289,717073.0,7.510418e+08,...,0.534338,-0.000310,-0.990648,0,0,1,2021-01-01 17:00:00,2.893401,1.0,19
2020-12-31 23:00:00,BTC,29123.18,29133.36,28936.00,29072.30,1486.628,2020-12-31,3.132289,717073.0,7.510418e+08,...,0.534338,0.002158,-0.988490,0,0,1,2021-01-01 17:00:00,2.410721,1.0,18


### Primary model 1

### Tripple Barrier method

### Meta Model

### Backtesting on 2021